# Import libraries

In [ ]:
import os
import numpy as np

from keras import callbacks
import keras_tuner as kt

from source_codes.data_aug import data_aug_with_random_contrast
from source_codes.ranking_model_prob import create_meta_network

# Prepare data

In [ ]:
# Configure question index
question_index = str(1)

# Load training data
training_data_path = os.path.join('data/datasets/train_data/draw_exclude')
img_left_training = np.load(os.path.join(training_data_path + '/train_left_duel_' + question_index + '.npy'), allow_pickle=True)
img_right_training = np.load(os.path.join(training_data_path + '/train_right_duel_' + question_index + '.npy'), allow_pickle=True)
label_training = np.load(os.path.join(training_data_path + '/train_label_duel_' + question_index + '.npy'), allow_pickle=True)

# Load validation data
validation_data_path = os.path.join('data/datasets/validation_data/draw_exclude')
img_left_validation = np.load(os.path.join(validation_data_path + '/val_left_duel_' + question_index + '.npy'), allow_pickle=True)
img_right_validation = np.load(os.path.join(validation_data_path + '/val_right_duel_' + question_index + '.npy'), allow_pickle=True)
label_validation = np.load(os.path.join(validation_data_path + '/val_label_duel_' + question_index + '.npy'), allow_pickle=True)

# Load test data
test_data_path = os.path.join('data/datasets/test_data/draw_exclude')
img_left_test = np.load(os.path.join(test_data_path + '/test_left_duel_' + question_index + '.npy'), allow_pickle=True)
img_right_test = np.load(os.path.join(test_data_path + '/test_right_duel_' + question_index + '.npy'), allow_pickle=True)
label_test = np.load(os.path.join(test_data_path + '/test_label_duel_' + question_index + '.npy'), allow_pickle=True)

## Augment data with random contrast
*This is a data augmentation method that raised by Hannick. It helps to improve the model performance.*
[Hannick's GitHub](https://github.com/Hannick5/UrbanPerception)

In [ ]:
img_left_training,img_right_training,label_training = data_aug_with_random_contrast(img_left_training,img_right_training, label_training)

# Define hptuning model

In [1]:
# Define model for hyperparameters tuning
hp = kt.HyperParameters()
hptuning_ranking_model = create_meta_network(
    dense_units=hp.Choice('dense_units', values=[32, 64, 128, 256, 512]),
    dropout_rate=hp.Float('dropout_rate', min_value=0.0, max_value=0.6, step=0.2),
    learning_rate=hp.Choice('learning_rate', values=[1e-4, 1e-5, 1e-6]),
    optimizer=hp.Choice('optimizer', values=['adam', 'rmsprop', 'sgd']),
    activation=hp.Choice('activation', values=['relu', 'tanh', 'sigmoid']),
    learning_rate_decay=hp.Choice('learning_rate_decay', values=[1e-3, 1e-4, 1e-5]),
    img_size=224,
    data_aug=True,
    weights=None
)

# Adopt Bayesian optimization
tuner = kt.BayesianOptimization(
    hptuning_ranking_model,
    objective=['val_loss', 'val_accuracy'],
    max_trials=20,
    executions_per_trial=4,
    overwrite=False,
    directory='model_results/Q2/hp_tuning',
    project_name="hptuning_ranking_230521",
)

# Tune hyperparameters

In [ ]:
# Employ early stopping to prevent overfitting
early_stopping = callbacks.EarlyStopping(
    monitor="val_loss",
    patience=10,
    mode='min',
    restore_best_weights=True
)

# Tune hyperparameters
tuner.search(
    x=[img_left_training, img_right_training],
    y=label_training,
    epochs=50,
    batch_size=32,
    validation_data=([img_left_validation, img_right_validation], label_validation),
    callbacks=[early_stopping]
)

# Obtain hptuning results

In [ ]:
# Display the tuning process summary
tuner.results_summary()

In [ ]:
# Save the best five hptuned model architectures
best_hps = tuner.get_best_hyperparameters(5)
save_dir = "model_results/Q2/hp_tuning/hptuning_ranking_230521"
for i in range(len(best_hps)):
    model = hptuning_ranking_model(best_hps[i])
    model.save(os.path.join(save_dir, 'best_model_' + str(i) + '_architecture.h5'))